In [2]:
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import csv

In [3]:
from platform import python_version

print(python_version())

3.8.5


In [5]:
keyword = 'Covid'
maxTweets = 30000

#Open/create a file to append data to
csvFile = open('result.csv', 'a', newline='', encoding='utf8')

#Use csv writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['id','date','tweet']) 

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + 'lang:en' + 'since:2020-06-01 until:2020-06-30 -filter:links -filter:replies').get_items()):
        if i > maxTweets :
            break      
        csvWriter.writerow([tweet.id, tweet.date, tweet.content])
csvFile.close()

In [10]:
maxTweets = 100

keyword = 'COVID
#place = '01fbe706f872cb32' # This geo_place string corresponds to Washington DC on twitter.

#Open/create a file to append data to
csvFile = open('result.csv', 'a', newline='', encoding='utf8')

#Use csv writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['id','date','tweet',]) 

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + ' lang:en since:2020-01-31 until:2020-11-03 -filter:links -filter:replies').get_items()):
        if i > maxTweets :
            break  
        csvWriter.writerow([tweet.id, tweet.date, tweet.content])
csvFile.close()